# Data in Decorators

Some decorators patterns rely on using variables inside the decorator function itself. This is not the same as variables inside the wrapper function.

e.g. You need to keep track of how many times a function is called using a single decorator for multiple functions

In [1]:
def call_count(func):
    data = {'calls': 0}
    def wrapper(*args, **kwargs):
        data['calls'] += 1
        print(f'Function {func.__name__} called {data["calls"]} times')
        return func(*args, **kwargs)
    return wrapper

Each time the function is called in the code, the wrapper function is what's actually executed

In [2]:
@call_count
def foo(x):
    return x + 10

@call_count
def bar(x):
    return x + 10

In [3]:
foo(10)

Function foo called 1 times


20

In [4]:
foo(10)

Function foo called 2 times


20

In [5]:
bar(50)

Function bar called 1 times


60

This is great, exactly what we want.

**The placement of data is important**. If we placed `data` outside of the decorator that means every decorator will share the same dictionary!

In [6]:
data = {'calls': 0}
def call_count(func):
    def wrapper(*args, **kwargs):
        data['calls'] += 1
        print(f'Function {func.__name__} called {data["calls"]} times')
        return func(*args, **kwargs)
    return wrapper


In [7]:
@call_count
def foo(x):
    return x + 10

@call_count
def bar(x):
    return x + 10

In [8]:
foo(10)

Function foo called 1 times


20

In [9]:
foo(10)

Function foo called 2 times


20

In [10]:
bar(10)

Function bar called 3 times


20

**Not the behaviour we want**. We could place the data dictionary inside the `wrapper` method but this will just reset the dictionary every time the function is called.

So what is actually happening in the example we get the desired behaviour? The decorator function itself is executed **exactly once** for **every** function it decorates. If you decorate N functions we get N different data dictionaries, each tied to one of the resulting decorated functions.

Now what if you want to peek at you data? You just simply assign at attribute to the wrapper function. A function in Python is just an object and you can add to new attributes by assigning them:

In [11]:
def call_count(func):
    data = {'calls': 0}
    def wrapper(*args, **kwargs):
        data['calls'] += 1
        return func(*args, **kwargs)
    wrapper.data = data
    return wrapper

In [12]:
@call_count
def foo(x):
    return x + 10

@call_count
def bar(x):
    return x + 10

In [13]:
foo(5)

15

In [14]:
foo(5)

15

In [15]:
bar(10)

20

In [16]:
foo.data

{'calls': 2}

In [17]:
bar.data

{'calls': 1}

You might be wonder why I'm using a dictionary for data inside the decorator rather than `calls = 0`. It's for the following reason:

In [18]:
def call_count(func):
    calls = 0
    def wrapper(*args, **kwargs):
        calls += 1
        return func(*args, **kwargs)
    wrapper.calls = calls
    return wrapper

In [19]:
@call_count
def foo(x):
    return x + 10

In [20]:
foo(10)

UnboundLocalError: local variable 'calls' referenced before assignment

It creates the error above. Why does this throw an error here but not with the dictionary. When the following code runs:
> count += 1

It's actually running and modifying the `count` variable
> count = count + 1

Whenever you modify (instead of just read) a variable that was created in a larger scope, Python requires you to declare that's what you actually want by either stating the variable is a `global` or `nonlocal`

It works with the dictionary becuase when we write:
> data['calls'] += 1

**this is not actually modifying the data**. Or rather it's not modifying the variable named data, which points to a dictionary object. Instead, the statement `data['calls'] += 1` invokes a method on the data object. This does change the state of the dictionary, but it doesn't make data point to a different dictionary. But `count += 1` makes `count` point to a different integer.

So how can we fix this? With the `non local` keyword.

In [21]:
def call_count(func):
    calls = 0
    def wrapper(*args, **kwargs):
        nonlocal calls
        calls += 1
        print(f'Number of calls: {calls}')
        return func(*args, **kwargs)
    return wrapper

In [22]:
@call_count
def foo(x):
    return x + 10

@call_count
def bar(x):
    return x + 10

In [23]:
foo(10)

Number of calls: 1


20

In [24]:
foo(10)

Number of calls: 2


20

Applying `nonlocal` gives calls variable a special scope that is part-way between local and global. Essentially Python will search for the nearest enclosing scope that defines a variable named calls.